In [40]:
from bert_score.scorer import BERTScorer
from lime.lime_text import LimeTextExplainer
import shap
import tqdm
import numpy as np
from utils import NpEncoder
import json
from scipy.stats.mstats import gmean
from numpy import mean
import pandas as pd
from evaluation_utils import evaluate_sentence_level, evaluate_word_level, validate_word_level_data

In [42]:
model_type = 'joeddav/xlm-roberta-large-xnli'
num_layers = 16
idf_sents = 0
scorer = BERTScorer(model_type=model_type, batch_size=64, num_layers=num_layers)

In [196]:
#Languague pairs for Eval4NLP are ro-en, et-en, de-zh, ru-en
SRC_LANG = 'et'
TGT_LANG = 'en'
SPLIT = 'test21'
data_dir = f'data/{SPLIT}/{SRC_LANG}-{TGT_LANG}-{SPLIT}'
src = [s.strip() for s in open(f'{data_dir}/{SPLIT}.src').readlines()]
tgt = [s.strip() for s in open(f'{data_dir}/{SPLIT}.mt').readlines()]
wor = [list(map(int, s.strip().split())) for s in open(f'{data_dir}/{SPLIT}.tgt-tags').readlines()]
sen = [float(s.strip()) for s in open(f'{data_dir}/{SPLIT}.da').readlines()]
assert len(src) == len(tgt) == len(wor) == len(sen)
dataset = {'src': src, 'tgt': tgt, 'word_labels': wor, 'sent_labels': sen}

In [197]:
bertscores ,hyp_explanations, src_explanations = scorer.score(dataset['tgt'], dataset['src'])

In [198]:
with open(f"limeresults_{SRC_LANG+TGT_LANG}_16_100.json") as f:
  results = json.load(f)

In [199]:
with open(f"shap_results_{SRC_LANG+TGT_LANG}.json") as f:
  results_shap = json.load(f)

In [200]:
def gen_mean(vals, p):
    p = float(p)
    return np.power(
        np.mean(
            np.power(
                np.array(vals, dtype=complex),
                p),
            axis=0),
        1 / p
    ).real

In [9]:
from numpy import min

In [187]:
sentence_scores = [ gen_mean([w[1] for w in s['1']],-5) for s in results]
sentence_scores = np.nan_to_num(sentence_scores,0)
cumscores =  np.add(bertscores[2]*0.2,sentence_scores*0.8)
value = evaluate_sentence_level(sen,cumscores)

Pearson correlation: 0.799


In [ ]:
dataframe = pd.DataFrame(columns=["src","tgt","p","w","corr"])

for i in range (-100,101,1):
  if i == 0:
    continue 
  p = i/10;
  sentence_scores = [ gen_mean([w[1] for w in s['1']],p) for s in results]
  sentence_scores = np.nan_to_num(sentence_scores,0)
  for i in range(11):
      cumscores =  np.add(bertscores[2]*i/10,sentence_scores*(1-i/10))
      print(f"I: {i}: ")
      value = evaluate_sentence_level(sen,cumscores)
      dataframe = pd.concat([dataframe,pd.DataFrame({"src":SRC_LANG, "tgt": TGT_LANG, "p":p, "w":i/10, "corr": value},index=[0])])

In [ ]:
#get p and w values for themaximum correlation
dataframe.iloc[dataframe["corr"].idxmax()]

In [ ]:
#plot different correlation for different p and w values
sns.relplot(x="p", y="corr",hue="w",height=10, data=dataframe.query('w==0|w==0.4 |w==0.5| w==0.6 | w==1'));